In [1]:
import os
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon, LineString
from shapely.ops import unary_union, voronoi_diagram, nearest_points
from shapely import affinity

from scipy.ndimage import label
from scipy.spatial import Voronoi
from scipy.optimize import minimize
from skimage import measure

import rasterio
from rasterio.transform import from_origin
from rasterstats import zonal_stats

import sklearn.cluster


In [2]:
raster_file = "data/flood_rasters/JamaicaJAM001RCP852100_epsg_32618_RP_100.tif"  #raster file containing flood data, used for analysis or overlays
inland_buffer_distance = 2  # Buffer distance (in kilometers) to expand the inital bounding boxes
depth_thresh = 0
eps_thresh = 3
return_period = 100


#Not defined by user
input_file = "data/Foundation_Data.gpkg"  # GeoPackage file with coastline edges and nodes
processing_file = "outputs/coastal_protection_processing_layers.gpkg"  # GeoPackage file to store intermediate processing layers
output_file = "outputs/Jamaica_coastal_protection_areas.gpkg"  # Final output file for coastal protection areas

coastline_edge_layer = gpd.read_file(input_file, layer="edges")  # Load edges as GeoDataFrame
coastline_node_layer = gpd.read_file(input_file, layer="nodes")  # Load nodes as GeoDataFrame

# coastline_edge_layer.to_file(output_file, layer="edges", driver="GPKG")  # Save edges to output file
# coastline_node_layer.to_file(output_file, layer="nodes", driver="GPKG")  # Save nodes to output file

jamaica_polygon_revised = gpd.read_file(input_file, layer="jam") 
jamaica_polygon_revised = jamaica_polygon_revised.to_crs(3448)
jamaica_polygon_revised["geometry"] = jamaica_polygon_revised.geometry.buffer(0)



In [3]:
def add_Layer_to_File(data, output_file, layer_name, driver):
    data.to_file(output_file, layer=layer_name, driver=driver)

In [4]:
# Adapted and modified from https://github.com/thomas-fred/jam-coastal-protection

def process_raster_to_clusters(threshold_m, eps, raster_file):
    """
    Processes a raster file to identify clusters of depth values using DBSCAN, 
    converts them to polygons, and returns a GeoDataFrame for further processing.

    Args:
        threshold_m (float): Minimum depth value to include pixels in the analysis.
        eps (float): DBSCAN epsilon parameter for cluster proximity.
        raster_file (str): File path to the input raster file.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame containing minimum enclosing polygons for clusters.
    """
    # Open the raster file and read the first band (depth values)
    raster = rasterio.open(raster_file)
    depth_m = raster.read(1)
    i, j = np.indices(depth_m.shape)  # Create row (i) and column (j) indices

    # Extract the transformation and CRS from the raster
    transform = from_origin(raster.bounds.left, raster.bounds.top, raster.res[0], raster.res[1])
    original_crs = raster.crs

    # Create a DataFrame and filter out pixels below the depth threshold
    df = pd.DataFrame(data={"i": i.ravel(), "j": j.ravel(), "depth_m": depth_m.ravel()})
    df = df[df["depth_m"] > threshold_m]  # Keep only pixels above the threshold

    if not df.empty:
        # Initialize the DBSCAN clustering algorithm and fit the filtered data
        db = sklearn.cluster.DBSCAN(eps=eps, min_samples=5)
        X = df.loc[:, ["i", "j"]].to_numpy()  # Extract pixel coordinates
        db.fit(X)

        # Retrieve cluster labels and count the number of clusters
        labels = db.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # Exclude noise points (-1)

        if n_clusters >= 2:
            # Create polygons from clusters and convert to a GeoDataFrame
            gdf = create_polygons_from_clusters(labels, depth_m.shape, transform, original_crs, df)

            # Reproject GeoDataFrame to EPSG:3448 coordinate reference system
            gdf = gdf.to_crs("EPSG:3448")

            # Generate minimum enclosing polygons for each cluster
            gdf = create_min_enclosing_polygons(gdf)
            return gdf


def create_polygons_from_clusters(labels, shape, transform, crs, df):
    """
    Converts DBSCAN cluster labels into polygons and organizes them in a GeoDataFrame.

    Args:
        labels (np.ndarray): Cluster labels assigned by DBSCAN (-1 for noise).
        shape (tuple): Shape of the raster (rows, cols).
        transform (Affine): Transformation to convert pixel indices to coordinates.
        crs (str): Coordinate reference system of the raster.
        df (pd.DataFrame): DataFrame containing pixel indices and other relevant data.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame containing polygons representing clusters.
    """
    # Create an array to store cluster labels mapped to raster pixels
    cluster_raster = np.full(shape, -1, dtype=np.int32)
    cluster_raster[df["i"], df["j"]] = labels

    polygons = []
    for cluster_label in np.unique(labels):
        if cluster_label == -1:
            continue  # Skip noise points

        # Identify the mask for the current cluster
        cluster_mask = cluster_raster == cluster_label
        contours = measure.find_contours(cluster_mask, level=0.5)

        # Convert contours to polygons
        for contour in contours:
            polygon_coords = [
                transform * (col, row) for row, col in contour
            ]
            polygon = Polygon(polygon_coords)
            if polygon.is_valid:
                polygons.append((cluster_label, polygon))

    # Create a GeoDataFrame from the polygons
    gdf = gpd.GeoDataFrame(polygons, columns=["cluster_label", "geometry"], crs=crs)
    return gdf


def create_min_enclosing_polygons(gdf):
    """
    Generates minimum enclosing polygons for each cluster in the GeoDataFrame.

    Args:
        gdf (gpd.GeoDataFrame): GeoDataFrame containing cluster polygons.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame with updated minimum enclosing polygons.
    """
    enclosing_polygons = []

    for cluster_label in gdf["cluster_label"].unique():
        # Combine all polygons for the same cluster label into a single geometry
        cluster_polygons = gdf[gdf["cluster_label"] == cluster_label]
        combined_geometry = unary_union(cluster_polygons.geometry)

        # Create the minimum enclosing polygon (remove self-intersections with buffer)
        enclosing_polygon = combined_geometry.buffer(0)
        enclosing_polygons.append((cluster_label, enclosing_polygon))

    # Create a new GeoDataFrame with the enclosing polygons
    gdf_enclosing = gpd.GeoDataFrame(enclosing_polygons, columns=["cluster_label", "geometry"], crs=gdf.crs)
    return gdf_enclosing


# Calling the function with appropriate parameters
dbscan_flood_areas = process_raster_to_clusters(
    threshold_m=depth_thresh,  # Minimum depth threshold
    eps=eps_thresh,            # DBSCAN epsilon value
    raster_file=raster_file    # Input raster file path
)

# Saving the output to a GeoPackage file
layer_name = f"dbscan_flood_areas_RP_{return_period}_eps_{eps_thresh}_thresh_{depth_thresh}"
add_Layer_to_File(dbscan_flood_areas, processing_file, layer_name, "GPKG")


In [5]:
def generate_polygons_along_edge(edge_layer, node_layer, buffer):
    """
    Generate polygons (bounding boxes) aligned along edges in a GeoDataFrame.

    For each edge, this function creates a rectangular bounding box centered on the edge's midpoint, 
    rotated to align with the edge's direction, and expanded vertically by the specified buffer.

    Parameters:
    - edge_layer (GeoDataFrame): GeoDataFrame containing edges (LineString) with 'from_id' and 'to_id' attributes.
    - node_layer (GeoDataFrame): GeoDataFrame containing nodes (Point) with 'node_id' attributes.
    - buffer (float): Additional height (vertical buffer) to be added to the bounding box, in the km.

    Returns:
    - GeoDataFrame: A GeoDataFrame containing rotated and translated bounding box polygons for each coastline segment.
    """
    # List to store the resulting bounding boxes for each edge
    bounding_boxes = []

    # Lists to store additional attributes for the resulting GeoDataFrame
    distances = []  # Store the length of each edge
    midpoints = []  # Store the midpoint of each edge
    angles = []     # Store the rotation angle for each edge

    # Iterate through each edge in the edge layer
    for _, edge in edge_layer.iterrows():
        # Retrieve the 'from_id' and 'to_id' for the current edge
        from_node_id = edge['from_id']
        to_node_id = edge['to_id']
        
        # Get the coordinates of the start and end nodes using their IDs
        from_node = node_layer[node_layer['node_id'] == from_node_id].geometry.iloc[0]
        to_node = node_layer[node_layer['node_id'] == to_node_id].geometry.iloc[0]

        # Skip edges where the start and end nodes are at the same location
        if (round(from_node.x, 5) == round(to_node.x, 5)) and (round(from_node.y, 5) == round(to_node.y, 5)):
            continue

        # Calculate the straight-line distance (diagonal) between the two nodes
        distance = from_node.distance(to_node)
        distances.append(distance)

        # Calculate the midpoint of the edge
        midpoint = calculate_midpoint(from_node, to_node)
        midpoints.append(midpoint)

        # Calculate the angle of the edge relative to the horizontal axis
        angle = calculate_angle(from_node, to_node)
        angles.append(angle)

        # Define the height of the bounding box (including buffer)
        min_y = min(from_node.y, to_node.y)
        max_y = max(from_node.y, to_node.y)
        height = max_y - min_y + (buffer * 1000)  # Apply buffer to the height

        # Define the width of the bounding box (distance between the two nodes)
        min_x = min(from_node.x, to_node.x)
        max_x = min_x + distance  # Adjust width to match edge length

        # Create an initial rectangular bounding box
        bounding_box = Polygon([
            (min_x, min_y),
            (min_x, min_y + height),  # Top-left corner
            (max_x, min_y + height),  # Top-right corner
            (max_x, min_y),           # Bottom-right corner
            (min_x, min_y)            # Close back to bottom-left
        ])

        # Translate the bounding box to center it on the edge's midpoint
        translated_bounding_box = translate_bounding_box(bounding_box, midpoint)

        # Rotate the bounding box to align with the edge's orientation
        rotated_bounding_box = rotate_bounding_box(translated_bounding_box, midpoint, angle)

        # Append the final bounding box to the list
        bounding_boxes.append(rotated_bounding_box)

    # Convert the list of bounding boxes into a GeoDataFrame
    bounding_boxes_gdf = gpd.GeoDataFrame(geometry=bounding_boxes, crs=edge_layer.crs)

    # Add additional attributes (distance, angle) to the resulting GeoDataFrame
    bounding_boxes_gdf['id'] = range(len(bounding_boxes_gdf))  # Unique ID for each polygon
    bounding_boxes_gdf['distance'] = distances  # Distance between the nodes
    bounding_boxes_gdf['angle'] = angles        # Angle of the edge in degrees

    return bounding_boxes_gdf

def calculate_midpoint(from_node, to_node):
    """
    Calculate the midpoint between two nodes.

    Parameters:
    - from_node (Point): The starting node as a Shapely Point.
    - to_node (Point): The ending node as a Shapely Point.

    Returns:
    - Point: The midpoint between the two nodes as a Shapely Point.
    """
    midpoint_x = (from_node.x + to_node.x) / 2
    midpoint_y = (from_node.y + to_node.y) / 2
    return Point(midpoint_x, midpoint_y)

def calculate_angle(from_node, to_node):
    """
    Calculate the angle of the line connecting two nodes relative to the horizontal axis.

    Parameters:
    - from_node (Point): The starting node as a Shapely Point.
    - to_node (Point): The ending node as a Shapely Point.

    Returns:
    - float: The angle in degrees, measured counterclockwise from the positive x-axis.
    """
    delta_x = to_node.x - from_node.x
    delta_y = to_node.y - from_node.y
    angle_rad = math.atan2(delta_y, delta_x)
    return math.degrees(angle_rad)

def translate_bounding_box(bounding_box, midpoint):
    """
    Translate a bounding box to center it on a specified midpoint.

    Parameters:
    - bounding_box (Polygon): The bounding box polygon to translate.
    - midpoint (Point): The target center point for translation.

    Returns:
    - Polygon: The translated bounding box polygon.
    """
    current_center_x = (bounding_box.bounds[0] + bounding_box.bounds[2]) / 2
    current_center_y = (bounding_box.bounds[1] + bounding_box.bounds[3]) / 2
    translate_x = midpoint.x - current_center_x
    translate_y = midpoint.y - current_center_y
    return affinity.translate(bounding_box, xoff=translate_x, yoff=translate_y)

def rotate_bounding_box(bounding_box, midpoint, angle):
    """
    Rotate a bounding box around a specified midpoint by a given angle.

    Parameters:
    - bounding_box (Polygon): The bounding box polygon to rotate.
    - midpoint (Point): The center point for rotation.
    - angle (float): The angle in degrees by which to rotate the bounding box.

    Returns:
    - Polygon: The rotated bounding box polygon.
    """
    return affinity.rotate(bounding_box, angle, origin=(midpoint.x, midpoint.y))

# Example usage
edge_polygons = generate_polygons_along_edge(
    edge_layer=coastline_edge_layer, 
    node_layer=coastline_node_layer,
    buffer=inland_buffer_distance,
)

# Save the output to a file
layer_name = f"edge_polygons_depth_{inland_buffer_distance}"
add_Layer_to_File(edge_polygons, processing_file, layer_name, "GPKG")


In [6]:
def add_max_zonal_stats(geopkg, raster_path, new_column_name):
    """
    Add a new column to a GeoDataFrame with the maximum raster value for each polygon,
    mimicking the functionality of the Zonal Statistics Tool in QGIS. 
    The GeoDataFrame is reprojected to match the raster CRS during the calculation 
    and reprojected back to its original CRS afterward.

    Parameters:
    - geopkg (GeoDataFrame): GeoDataFrame containing polygons for which zonal statistics will be calculated.
    - raster_path (str): Path to the raster file used for calculating zonal statistics.
    - new_column_name (str): Name of the new column to store the maximum raster value for each polygon.

    Returns:
    - GeoDataFrame: The updated GeoDataFrame with the new column containing maximum raster values.
    """
    # Save the original CRS (Coordinate Reference System) of the GeoDataFrame for re-projection later
    original_crs = geopkg.crs

    # Open the raster file to retrieve its CRS
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs

    # Reproject the GeoDataFrame to match the raster CRS if their CRS do not align
    if geopkg.crs != raster_crs:
        geopkg = geopkg.to_crs(raster_crs)

    # Ensure all geometries are valid (use a buffer of 0 as a fix for invalid geometries)
    geopkg["geometry"] = geopkg["geometry"].buffer(0)

    # Calculate zonal statistics to determine the maximum raster value within each polygon
    stats = zonal_stats(
        geopkg,          # GeoDataFrame containing polygons
        raster_path,     # Path to the raster file
        stats="max",     # Calculate the maximum value for each zone
        geojson_out=False,  # Do not output results in GeoJSON format
        nodata=-9999     # Value to treat as NoData in the raster (adjust as necessary)
    )

    # Extract the maximum raster values from the zonal statistics results and add to a new column
    geopkg[new_column_name] = [
        stat["max"] if stat["max"] is not None else None  # Add max value or None if no data exists
        for stat in stats
    ]

    # Reproject the GeoDataFrame back to its original CRS
    if geopkg.crs != original_crs:
        geopkg = geopkg.to_crs(original_crs)

    return geopkg  # Return the updated GeoDataFrame with the new column


# Call the function to calculate and add the maximum flood height from the raster
edge_polygons = add_max_zonal_stats(
    geopkg=edge_polygons,
    raster_path=raster_file,
    new_column_name="max_flood_height"
)

# Save the updated GeoDataFrame back to the file, ensuring CRS consistency
layer_name = f"edge_polygons_depth_{inland_buffer_distance}"
add_Layer_to_File(edge_polygons, processing_file, layer_name, "GPKG")


In [7]:
def join_polygons_by_flood_height(gdf, flood_height_threshold, area_limit, group_size):
    """
    Groups polygons in a GeoDataFrame based on their maximum flood height, with additional grouping 
    conditions based on area size and maximum group size. 
    
    Polygons are joined together if their flood height exceeds a specified threshold or if their combined area
    exceeds a specified limit.

    Parameters:
    - gdf (GeoDataFrame): Input GeoDataFrame containing polygons and their associated maximum flood heights
    - flood_height_threshold (float): The flood height threshold above which polygons are grouped together
    - area_limit (float): The area limit for groups; groups with areas exceeding this limit are finalized
    - group_size (int): The  minimum number of polygons that can make up a group

    Returns:
    - GeoDataFrame: A new GeoDataFrame containing the grouped polygons, with a new 'max_flood_height' column
    """
    
    # Initialize lists to store results for each group
    groups = []  # List of combined polygons (grouped geometries)
    heights = []  # List of maximum flood heights for each group
    ids = []  # List of original polygon IDs for each group
    
    # Initialize variables to track the current group being processed
    current_group = []  # List of polygons in the current group
    current_heights = []  # List of max flood heights for the current group
    current_ids = []  # List of original IDs for the current group
    is_below_threshold = None  # Flag indicating whether the current polygon is below the threshold

    # Iterate over each polygon in the GeoDataFrame
    for index, row in gdf.iterrows():
        if row["max_flood_height"] > flood_height_threshold:
            # If the polygon's flood height exceeds the threshold, handle grouping
            if is_below_threshold is True or (current_group and unary_union(current_group).area > area_limit):
                # Finalize the current group if necessary (area limit exceeded or threshold state changed)
                if len(current_group) < group_size and groups:
                    # Add the current polygon to the previous group (if the group reached max size)
                    groups[-1] = unary_union([groups[-1], current_group[0]]).convex_hull
                    heights[-1] = max(heights[-1], current_heights[0])  # Update max height
                    ids[-1].extend(current_ids)  # Add original IDs to the group
                else:
                    # Finalize the group normally by combining geometries
                    combined_geometry = unary_union(current_group)
                    groups.append(combined_geometry.convex_hull)
                    heights.append(max(current_heights))  # Store the max flood height for the group
                    ids.append(current_ids)  # Store the original IDs for the group

                # Reset the current group and associated data for the next set of polygons
                current_group = []
                current_heights = []
                current_ids = []

            # Switch to above-threshold mode
            is_below_threshold = False
            current_group.append(row["geometry"])  # Add the polygon's geometry to the current group
            current_heights.append(row["max_flood_height"])  # Add max flood height to the current group
            current_ids.append(row["id"])  # Add the polygon's ID to the current group

        else:  # Polygons below or equal to the flood height threshold
            # If the polygon is below the threshold, handle the grouping similarly
            if is_below_threshold is False or (current_group and unary_union(current_group).area > area_limit):
                # Finalize the current group if necessary (area limit exceeded or threshold state changed)
                if len(current_group) < group_size and groups:
                    # Add the current polygon to the previous group
                    groups[-1] = unary_union([groups[-1], current_group[0]]).convex_hull
                    heights[-1] = max(heights[-1], current_heights[0])  # Update max height
                    ids[-1].extend(current_ids)  # Add original IDs to the group
                else:
                    # Finalize the group normally by combining geometries
                    combined_geometry = unary_union(current_group)
                    groups.append(combined_geometry.convex_hull)
                    heights.append(max(current_heights))  # Store the max flood height for the group
                    ids.append(current_ids)  # Store the original IDs for the group

                # Reset the current group and associated data for the next set of polygons
                current_group = []
                current_heights = []
                current_ids = []

            # Switch to below-threshold mode
            is_below_threshold = True
            current_group.append(row["geometry"])  # Add the polygon's geometry to the current group
            current_heights.append(row["max_flood_height"])  # Add max flood height to the current group
            current_ids.append(row["id"])  # Add the polygon's ID to the current group

    # Finalize the last group if any polygons remain
    if current_group:
        if len(current_group) < group_size and groups:
            # Add the current polygon to the previous group (if group size exceeded)
            groups[-1] = unary_union([groups[-1], current_group[0]]).convex_hull
            heights[-1] = max(heights[-1], current_heights[0])  # Update max height
            ids[-1].extend(current_ids)  # Add original IDs to the group
        else:
            # Finalize the group normally by combining geometries
            combined_geometry = unary_union(current_group)
            groups.append(combined_geometry.convex_hull)
            heights.append(max(current_heights))  # Store max flood height
            ids.append(current_ids)  # Store original IDs

    # Convert the grouped data back into a GeoDataFrame
    result_gdf = gpd.GeoDataFrame(
        {
            "geometry": gpd.GeoSeries(groups),  # Combined geometries for each group
            "max_flood_height": heights,  # Max flood heights for each group
            "original_ids": ids,  # Original IDs for each group
        }
    )

    # Add a unique ID field to the result GeoDataFrame
    result_gdf["id"] = range(len(result_gdf))

    # Set the CRS (Coordinate Reference System) to match the original GeoDataFrame
    result_gdf.set_crs(gdf.crs, inplace=True)

    return result_gdf  # Return the GeoDataFrame with grouped polygons


def order_edges_around_island(edge_layer):
    """
    Order edges around an island, starting with edge_0 and following the
    connections based on from_id and to_id.

    Parameters:
    - edge_layer (GeoDataFrame): GeoDataFrame containing edges with 'id', 'from_id', and 'to_id' columns

    Returns:
    - List[str]: List of associated bounding box IDs in the order the edges are traced around the island
    """
    # Initialize the ordered list with the first edge (edge_0)
    ordered_edges = []

    # Create a dictionary to lookup edges by their 'from_id' for efficient access
    edges_by_from_id = {
        edge['from_id']: edge
        for _, edge in edge_layer.iterrows()
    }

    # Start with the edge having 'id' equal to 'edge_0'
    current_edge = edge_layer[edge_layer['id'] == 'edge_0'].iloc[0]
    ordered_edges.append(current_edge['rectangle_id'])  # Append the first edge's ID

    # Track the starting node to detect when we've completed a loop
    start_node = current_edge['from_id']

    # Continue looping through the edges, following the 'to_id' until we return to the starting node
    while True:
        # Find the next edge based on the current edge's 'to_id'
        next_from_id = current_edge['to_id']

        # Break the loop if we return to the starting node (complete the loop)
        if next_from_id == start_node:
            break

        # Look up the next edge using the 'from_id' as the key in the dictionary
        current_edge = edges_by_from_id[next_from_id]
        ordered_edges.append(current_edge['rectangle_id'])  # Add the next edge's ID to the ordered list

    return ordered_edges  # Return the ordered list of edge IDs


# Call the function to get the ordered list of edge IDs
ordered_edge_ids = order_edges_around_island(
    edge_layer = coastline_edge_layer
)

# Filter the original polygon layer to include only the ordered edge IDs
ordered_edge_ids = [eid for eid in ordered_edge_ids if eid in edge_polygons["id"].values]
input_polygons = edge_polygons.set_index("id").loc[ordered_edge_ids].reset_index()

# Parameters for grouping polygons
flood_height_threshold = 0  # Flood height threshold above which polygons will be grouped
area_limit = 8_000_000  # Area limit for group finalization
group_size = 2  # Minimum number of polygonsneeded to make up a group

# Call the function to join polygons based on flood height and area conditions
joined_polygons = join_polygons_by_flood_height(
    gdf = input_polygons, 
    flood_height_threshold = flood_height_threshold, 
    area_limit = area_limit, 
    group_size = group_size
)

print (len(joined_polygons))
# Define layer name for the final output
layer_name = f"joined_edge_polygons_flood_{flood_height_threshold}_area_{area_limit}_group_{group_size}"

# Save the result back to a file, ensuring CRS consistency
add_Layer_to_File(joined_polygons, processing_file, layer_name, "GPKG")


296


In [8]:
def merge_overlapping_polygons(gdf, overlap_threshold):
    """
    Iteratively merges overlapping polygons that overlap by more than the specified threshold.
    
    Args:
        gdf (GeoDataFrame): GeoDataFrame containing polygons with 'geometry' and 'max_flood_height' fields.
        overlap_threshold (float): The minimum percentage of overlap required to trigger a merge (e.g., 0.35 for 35% overlap).
    
    Returns:
        GeoDataFrame: A new GeoDataFrame with merged polygons until no pair overlaps by more than the threshold.
    """
    
    # Ensure that the CRS (Coordinate Reference System) is defined for the input data
    if gdf.crs is None:
        gdf.set_crs("EPSG:3097", inplace=True)  # Set CRS to Jamaica Metric Grid, adjust if necessary

    # Helper function to perform one pass of merging overlapping polygons
    def merge_once(gdf, overlap_threshold):
        """
        Performs one pass over the GeoDataFrame to merge overlapping polygons.
        Merges polygons whose intersection area exceeds the given overlap threshold.
        
        Args:
            gdf (GeoDataFrame): GeoDataFrame to process.
            overlap_threshold (float): The overlap threshold to trigger a merge.
        
        Returns:
            GeoDataFrame: A new GeoDataFrame with merged polygons from this pass.
        """
        merged = []  # List to hold merged geometries
        indices_to_merge = set()  # Set to keep track of merged polygons

        # Loop through all polygons in the GeoDataFrame to check for overlaps
        for i, geom1 in enumerate(gdf.geometry):
            if i in indices_to_merge:
                continue  # Skip geometries that are already merged

            for j, geom2 in enumerate(gdf.geometry):
                if i >= j or j in indices_to_merge:
                    continue  # Avoid redundant checks and already-merged geometries

                # Check if the two polygons intersect
                if geom1.intersects(geom2):
                    intersection = geom1.intersection(geom2)
                    if intersection.is_empty:
                        continue  # Skip if the intersection is empty

                    # Calculate the area of the intersection and compare with the polygons' areas
                    area1 = geom1.area
                    area2 = geom2.area
                    intersection_area = intersection.area

                    # Check if the overlap exceeds the threshold
                    if (intersection_area / min(area1, area2)) > overlap_threshold:
                        # Merge the two polygons by creating a convex hull around the union
                        new_geom = unary_union([geom1, geom2]).convex_hull
                        new_height = max(gdf.loc[i, "max_flood_height"], gdf.loc[j, "max_flood_height"])

                        # Store the merged geometry and its max flood height
                        merged.append({
                            "geometry": new_geom,
                            "max_flood_height": new_height
                        })
                        indices_to_merge.update([i, j])  # Mark the merged polygons

                        break  # Stop checking other polygons for the current geometry once merged

        # Retain the geometries that were not merged
        non_merged = gdf.loc[~gdf.index.isin(indices_to_merge)]

        # Combine the merged geometries with the non-overlapping ones
        merged_gdf = gpd.GeoDataFrame(
            {
                "geometry": [m["geometry"] for m in merged],
                "max_flood_height": [m["max_flood_height"] for m in merged],
            }
        )

        # Ensure CRS is explicitly set for the merged geometries
        merged_gdf.set_crs(gdf.crs, inplace=True)

        # Concatenate the non-overlapping geometries with the merged ones
        final_gdf = gpd.GeoDataFrame(pd.concat([non_merged, merged_gdf], ignore_index=True))

        # Set CRS again after concatenation to ensure consistency
        final_gdf.set_crs(gdf.crs, inplace=True)

        return final_gdf

    # Iteratively merge polygons until no more overlaps are found
    prev_gdf = gdf
    while True:
        new_gdf = merge_once(prev_gdf, overlap_threshold)
        if len(new_gdf) == len(prev_gdf):  # No changes (no more overlaps)
            break
        prev_gdf = new_gdf  # Update the GeoDataFrame for the next iteration

    # Ensure final CRS consistency
    prev_gdf.set_crs(gdf.crs, inplace=True)

    # Reassign unique IDs starting from 1 for the merged polygons
    prev_gdf = prev_gdf.reset_index(drop=True)  # Reset the index
    prev_gdf['id'] = prev_gdf.index + 1  # Assign new unique IDs based on the index

    return prev_gdf

overlap = 0.39

# Apply the merging function to resolve overlaps
s_cape_flood_areas = merge_overlapping_polygons(
    gdf = joined_polygons, 
    overlap_threshold = overlap
)

# Create a layer name dynamically for the merged flood areas
layer_name = f"scape_flood_areas_flood_{flood_height_threshold}_area_{area_limit}_group_{group_size}_overlap_{overlap}"

# Add the merged polygons layer to the file
add_Layer_to_File(s_cape_flood_areas, processing_file, layer_name, "GPKG")


In [9]:
def create_trimmed_convex_hull(polygons):
    """
    Create a convex hull encompassing the given polygons by computing the convex hull
    from their union.
    
    Args:
        polygons (list): A list of geometries (polygons) to create the convex hull around.
    
    Returns:
        geometry: The convex hull geometry encompassing all the input polygons.
    """
    # Combine the polygons and compute the convex hull of their union
    combined = unary_union(polygons)  # Union of all polygons
    convex_hull = combined.convex_hull  # Convex hull of the union

    return convex_hull

def connect_polygons_with_convex_hull(flood_polygons):
    """
    Generate a trimmed convex hull connecting the given polygons and save the result to a new GeoPackage.
    
    Args:
        flood_polygons (GeoDataFrame): GeoDataFrame containing the polygons to connect with a convex hull.
    
    Returns:
        GeoDataFrame: A GeoDataFrame containing the trimmed convex hull as a new polygon.
    """
    # Use the provided GeoDataFrame of flood polygons
    gdf = flood_polygons

    # In this case, all polygons are selected, but this can be modified for specific selections
    selected_gdf = gdf

    # Ensure the geometry column exists and convert to a list of geometries
    polygons = selected_gdf.geometry.tolist()  # List of geometries for hull creation

    # Create the trimmed convex hull using the helper function
    trimmed_hull = create_trimmed_convex_hull(polygons)

    # Prepare a new GeoDataFrame for the trimmed convex hull
    new_row = gpd.GeoDataFrame({
        'id': ['trimmed_hull'],  # Assign an ID to the new hull
        'geometry': [trimmed_hull]  # The geometry of the convex hull
    }, crs=gdf.crs)  # Ensure the new row uses the same CRS as the input GeoDataFrame

    return new_row

# Generate the convex hull for the flood polygons
jamaica_convex_hull = connect_polygons_with_convex_hull(
    flood_polygons = s_cape_flood_areas
)

# Define the layer name for the convex hull and save it to the GeoPackage
layer_name = "jamaica_convex"
add_Layer_to_File(jamaica_convex_hull, processing_file, layer_name, "GPKG")


In [10]:
def create_voronoi(jamaica_polygon, jam_contour, coastline_polygons):
    """
    Create Voronoi tessellation based on the centroids of smaller polygons (coastline),
    clipped to the boundary of a larger polygon (Jamaica's convex hull).
    
    Args:
        jamaica_polygon (GeoDataFrame): The larger polygon (e.g., Jamaica's convex hull).
        jam_contour (GeoDataFrame): A GeoDataFrame containing the exact contour of the Jamaica.
        coastline_polygons (GeoDataFrame): The smaller polygons (e.g., coastline polygons) for which Voronoi regions will be calculated.

    Returns:
        GeoDataFrame: Clipped Voronoi polygons inside the larger polygon with associated centroid IDs.
        GeoDataFrame: Centroids of the smaller polygons used for Voronoi tessellation with a 'c_id' column.
    """
    # Load the larger polygon (assumed to be Jamaica's convex hull)
    convex_gdf = jamaica_polygon
    larger_polygon = convex_gdf.iloc[0].geometry  # Assuming the first row contains the larger polygon

    # Load the coastline polygons (smaller polygons)
    gdf = coastline_polygons
    smaller_polygons = gdf

    # Ensure the CRS is consistent for all GeoDataFrames
    smaller_polygons.crs = gdf.crs  # Ensure the coordinate reference system is consistent

    #-------------------------------------------------
    # Extract centroids of the smaller polygons to serve as Voronoi seed points
    centroids = smaller_polygons.geometry.centroid

    # Add a 'c_id' column to the centroids, which identifies the original polygon
    centroid_ids = smaller_polygons['id'] if 'id' in smaller_polygons.columns else range(len(centroids))
    centroid_gdf = gpd.GeoDataFrame({'c_id': centroid_ids}, geometry=centroids, crs=smaller_polygons.crs)

    # Function to adjust points to be within or touching a larger polygon
    def adjust_point_to_polygon(point, polygon):
        """
        Adjust the point to be within or touching the boundary of the larger polygon.
        If the point is outside, it will be moved to the nearest boundary point.
        
        Args:
            point (Point): The point (centroid) to adjust.
            polygon (Polygon): The larger polygon representing Jamaica's exact shape.
        
        Returns:
            Point: The adjusted point that is within or on the boundary of the polygon.
        """
        if polygon.contains(point) or polygon.touches(point):
            return point
        else:
            # Find the nearest point on the boundary
            nearest_boundary_point = nearest_points(point, polygon.boundary)[1]
            # Move the point slightly closer to the boundary
            adjusted_point = Point(
                point.x + (nearest_boundary_point.x - point.x),
                point.y + (nearest_boundary_point.y - point.y)
            )
            # Ensure the adjusted point is within or touching the polygon
            if polygon.contains(adjusted_point) or polygon.touches(adjusted_point):
                return adjusted_point
            # As a fallback, snap directly to the nearest boundary point
            return nearest_boundary_point

    # Adjust centroids to be within or touching the larger polygon (Jamaica's boundary)
    adjusted_centroids = [adjust_point_to_polygon(pt, jam_contour.iloc[0].geometry) for pt in centroids]

    # Convert the adjusted centroids back to a GeoSeries for consistent geometry
    adjusted_centroids_gs = gpd.GeoSeries(adjusted_centroids, crs=centroid_gdf.crs)

    # Update the GeoDataFrame with adjusted centroids
    centroid_gdf.geometry = adjusted_centroids_gs

    # Create a MultiPoint object from the adjusted centroids for Voronoi tessellation
    seed_points = MultiPoint(adjusted_centroids)

    # Perform Voronoi tessellation using the adjusted centroids as seed points
    voronoi = voronoi_diagram(seed_points)

    #-------------------------------------------------
    # Clip the Voronoi regions to the boundary of the larger polygon (Jamaica's convex hull)
    clipped_regions = [region.intersection(larger_polygon) for region in voronoi.geoms]

    # Filter out invalid or empty regions, keeping only valid polygons
    valid_regions = [region for region in clipped_regions if not region.is_empty and isinstance(region, Polygon)]

    # Match centroids to Voronoi regions based on proximity (the closest centroid to each Voronoi region)
    associated_centroid_ids = []
    for region in valid_regions:
        # Find the centroid nearest to the region's centroid
        region_centroid = region.centroid
        nearest_centroid = centroids.distance(region_centroid).idxmin()
        associated_centroid_ids.append(centroid_gdf.loc[nearest_centroid, 'c_id'])

    # Create a GeoDataFrame for the clipped Voronoi regions with associated centroid IDs
    clipped_gdf = gpd.GeoDataFrame(
        {'id': range(len(valid_regions)),  # Assign a unique ID to each Voronoi region
         'centroid_id': associated_centroid_ids},  # Store the associated centroid ID for each region
        geometry=valid_regions,
        crs=smaller_polygons.crs  # Use the same CRS as the smaller polygons
    )

    # Return both the clipped Voronoi regions and the centroids GeoDataFrame
    return clipped_gdf, centroid_gdf

# Load contour and coastline polygons from the input files
jam_contour = gpd.read_file(input_file, layer="jam") 

# Create the Voronoi polygons clipped to Jamaica's convex hull
voronoi_polygons, voronoi_centroid_points = create_voronoi(
    jamaica_polygon = jamaica_convex_hull, 
    jam_contour = jam_contour,
    coastline_polygons = s_cape_flood_areas,
)

# Save the Voronoi polygons and centroid points to a GeoPackage
layer_name = "voronoi_polygons"
add_Layer_to_File(voronoi_polygons, processing_file, layer_name, "GPKG")

layer_name = "voronoi_centroid_points"
add_Layer_to_File(voronoi_centroid_points, processing_file, layer_name, "GPKG")


In [21]:
# Adapted and modified from https://github.com/thomas-fred/jam-coastal-protection

def combine_floods_within_voronoi(voronoi_polygons: gpd.GeoDataFrame,flood_polygons: gpd.GeoDataFrame,voronoi_points: gpd.GeoDataFrame,max_distance: float) -> gpd.GeoDataFrame:
    """
    Combines all parts of flood polygons that intersect with each Voronoi polygon,
    but remove any parts whose centroids are farther than a given distance from
    the corresponding Voronoi point.

    Args:
        voronoi_polygons (gpd.GeoDataFrame): GeoDataFrame containing the Voronoi polygons.
        flood_polygons (gpd.GeoDataFrame): GeoDataFrame containing the flood polygons to combine.
        voronoi_points (gpd.GeoDataFrame): GeoDataFrame containing the Voronoi points with a 'fid' column that matches the 'centroid_id' column.
        max_distance (float): Maximum allowed distance (in the same CRS units) between a polygon's centroid and the corresponding Voronoi point.

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame containing the combined flood polygons for each Voronoi polygon,
                           with the flood areas merged based on their proximity to the Voronoi points.
    """
    combined_polygons = []  # List to store resulting geometries
    voronoi_ids = []  # List to store Voronoi IDs (optional, for tracking)

    # Loop through each Voronoi polygon
    for voronoi_index, voronoi in voronoi_polygons.iterrows():
        voronoi_geom = voronoi.geometry
        centroid_id = voronoi["centroid_id"]  # Get the Voronoi polygon's centroid ID

        # Get the corresponding Voronoi point based on the centroid ID
        voronoi_point = voronoi_points[voronoi_points["c_id"] == centroid_id]
        if voronoi_point.empty:
            continue  # Skip if no matching Voronoi point is found

        voronoi_point_geom = voronoi_point.geometry.iloc[0]

        # Find all flood polygons that intersect with the current Voronoi polygon
        intersecting_floods = flood_polygons[flood_polygons.intersects(voronoi_geom)]

        if not intersecting_floods.empty:
            # Combine the intersecting flood polygons within the Voronoi polygon
            combined_geom = intersecting_floods.intersection(voronoi_geom).union_all()

            # Filter out parts of the combined geometry that are too far from the Voronoi point
            if isinstance(combined_geom, MultiPolygon):
                # If the combined geometry is a MultiPolygon, check each part
                filtered_parts = [
                    part for part in combined_geom.geoms
                    if part.centroid.distance(voronoi_point_geom) <= max_distance
                ]
                combined_geom = MultiPolygon(filtered_parts) if filtered_parts else None
            elif combined_geom.centroid.distance(voronoi_point_geom) > max_distance:
                # If the combined geometry is a single polygon, check its centroid distance
                combined_geom = None

            if combined_geom:
                # Append the filtered geometry and its Voronoi index
                combined_polygons.append(combined_geom)
                voronoi_ids.append(voronoi_index)

    # Create a GeoDataFrame for the combined polygons
    result_gdf = gpd.GeoDataFrame({
        "voronoi_id": voronoi_ids,  # Include the Voronoi IDs
        "geometry": combined_polygons  # Include the geometries
    }, crs=voronoi_polygons.crs)  # Set CRS to match Voronoi polygons

    return result_gdf  # Return the combined flood polygons GeoDataFrame


def linestring_intersect_polygons(default_inland_distance: float, coast: gpd.GeoDataFrame, polygons: gpd.GeoDataFrame, flood_polygons: gpd.GeoDataFrame) -> (gpd.GeoDataFrame, gpd.GeoDataFrame):
    """
    Sections the coastline based on intersections with each Voronoi polygon and individually 
    buffers the new linestring segments until they completely enclose the flood polygon 
    within the corresponding Voronoi section.

    Also returns a GeoDataFrame of intersections between the linestring and Voronoi polygons before any buffering.

    Args:
        default_inland_distance (float): Buffer radius in kilometers.
        coast (gpd.GeoDataFrame): GeoDataFrame containing the coastline geometry to be buffered.
        polygons (gpd.GeoDataFrame): GeoDataFrame of Voronoi polygons to check intersections.
        flood_polygons (gpd.GeoDataFrame): GeoDataFrame of flood polygons with a 'voronoi_id' column to associate with Voronoi polygons.

    Returns:
        tuple: 
            - GeoDataFrame containing the intersections of the linestring and Voronoi polygons before buffering.
            - GeoDataFrame containing the final intersected and buffered geometries, with Voronoi IDs.
    """
    # Prepare lists to store intersections and final buffered geometries
    initial_intersections = []
    final_intersections = []

    # Create a GeoDataFrame containing the union of all coastline geometries
    linestring = gpd.GeoDataFrame({"id": [0], "geometry": [coast.geometry.union_all()]})
    linestring.crs = coast.crs  # Set CRS to match the coast's CRS

    # Iterate over each Voronoi polygon
    for idx, poly in polygons.iterrows():
        # Check if the linestring intersects with the Voronoi polygon
        if linestring.geometry.iloc[0].intersects(poly.geometry):
            # Perform the intersection between the linestring and the Voronoi polygon
            intersection = linestring.geometry.iloc[0].intersection(poly.geometry)
            
            # Append the raw intersection with the associated Voronoi ID
            initial_intersections.append({
                "geometry": intersection,
                "voronoi_id": idx
            })

            # Buffer the intersection piece separately based on the provided inland distance
            buffered_intersection = intersection.buffer(default_inland_distance * 1_000)
            
            # Find the corresponding flood polygon that has the same 'voronoi_id'
            flood_polygon = flood_polygons[flood_polygons['voronoi_id'] == idx]
            
            if not flood_polygon.empty:
                flood_geometry = flood_polygon.geometry.iloc[0]
                
                # Start iteratively buffering the intersection from 0.5 km and increase by 0.1 km
                current_buffer_radius = 0.5 * 1_000  # Start with a buffer radius of 0.5 km
                while not buffered_intersection.contains(flood_geometry):
                    # Increase buffer by 0.1 km at each iteration until it contains the flood geometry
                    current_buffer_radius += 0.1 * 1_000
                    buffered_intersection = intersection.buffer(current_buffer_radius)
            
            # Perform the final intersection with the flood polygon
            final_intersection = buffered_intersection.intersection(poly.geometry)
            
            # Append the final intersection with its associated Voronoi ID to the list
            final_intersections.append({
                "geometry": final_intersection,
                "voronoi_id": idx
            })
    
    # Create GeoDataFrames for initial and final intersections
    initial_intersections_gdf = gpd.GeoDataFrame(initial_intersections, crs=polygons.crs)
    final_intersections_gdf = gpd.GeoDataFrame(final_intersections, crs=polygons.crs)

    return initial_intersections_gdf, final_intersections_gdf



# Combine flood polygons for each Voronoi polygon using the function above
dbscan_voronoi_intersection = combine_floods_within_voronoi(
    voronoi_polygons = voronoi_polygons,  # Voronoi polygons GeoDataFrame
    flood_polygons = dbscan_flood_areas,  # Flood polygons GeoDataFrame
    voronoi_points = voronoi_centroid_points,  # Voronoi centroids GeoDataFrame
    max_distance = 8_000  # Maximum distance (in km) for filtering based on proximity to Voronoi points
)

# Add the resulting dbscan_voronoi_intersection layer to a processing file (GeoPackage format)
layer_name = "dbscan_voronoi_intersection"
add_Layer_to_File(dbscan_voronoi_intersection, processing_file, layer_name, "GPKG")

# Calculate initial and final intersections
flood_protection_coastline, flood_protection_areas = linestring_intersect_polygons(
    default_inland_distance=0.5,  # Buffer radius in kilometers
    coast=coastline_edge_layer,  # Coastline GeoDataFrame
    polygons=voronoi_polygons,  # Voronoi polygons GeoDataFrame
    flood_polygons=dbscan_voronoi_intersection,  # Flood polygons GeoDataFrame from previous step
)

# Save the raw intersections to the processing file
add_Layer_to_File(flood_protection_coastline, processing_file, "flood_protection_coast", "GPKG")

# Save the final flood protection areas to the processing file
add_Layer_to_File(flood_protection_areas, processing_file, "flood_protection_areas", "GPKG")



In [22]:
def clean_MultiPolygons(gdf):
    """
    This function processes a GeoDataFrame (gdf) containing geometries of type Polygon or MultiPolygon.
    It selects the largest Polygon from each MultiPolygon based on area, and leaves individual Polygons unchanged.
    
    Args:
        gdf (GeoDataFrame): A GeoDataFrame containing geometries that can be either Polygons or MultiPolygons.
        
    Returns:
        GeoDataFrame: A modified GeoDataFrame with only the largest Polygon selected from each MultiPolygon.
    """
    
    def largest_polygon(geom):
        """
        This helper function determines the largest polygon in a MultiPolygon, 
        or returns the Polygon as-is if it is not a MultiPolygon.
        
        Args:
            geom: The geometry object (either a Polygon or MultiPolygon).
        
        Returns:
            Polygon: The largest Polygon from a MultiPolygon, or the Polygon as-is if it is already a single Polygon.
        """
        if isinstance(geom, MultiPolygon):
            # If the geometry is a MultiPolygon, find and return the largest Polygon based on area
            return max(geom.geoms, key=lambda poly: poly.area)  # Access the 'geoms' attribute of the MultiPolygon
        elif isinstance(geom, Polygon):
            # If the geometry is already a Polygon, return it as it is
            return geom
        else:
            # If the geometry is neither a Polygon nor a MultiPolygon, return it unchanged or handle it differently
            return geom

    # Apply the 'largest_polygon' function to each geometry in the GeoDataFrame
    # This updates the 'geometry' column in the GeoDataFrame with the cleaned geometries
    gdf['geometry'] = gdf['geometry'].apply(largest_polygon)

    # Return the modified GeoDataFrame with cleaned geometries
    return gdf


# Apply the 'clean_MultiPolygons' function to the GeoDataFrame containing flood protection areas
final_coastal_protection_area = clean_MultiPolygons(
    gdf = flood_protection_areas  # GeoDataFrame containing flood protection polygons or multipolygons
)

# Process the cleaned GeoDataFrame by adding the maximum zonal statistics from the raster data
final_coastal_protection_area = add_max_zonal_stats(
    geopkg = final_coastal_protection_area,  # The cleaned GeoDataFrame
    raster_path = raster_file,  # Path to the raster file containing flood height data
    new_column_name= "max_flood_height"  # New column to store the maximum flood height value
)

# Clip the cleaned and processed GeoDataFrame to the boundaries of a specific polygon (e.g., a country or region)
# final_coastal_protection_area = gpd.clip(final_coastal_protection_area, jamaica_polygon_revised)


# Define the layer name to be used for storing the final protection area in a GeoPackage
layer_name = "final_protection_area"

# Add the final processed GeoDataFrame to the processing GeoPackage
add_Layer_to_File(final_coastal_protection_area, processing_file, layer_name, "GPKG")

# Also save the final protection area to the output file in GeoPackage format
add_Layer_to_File(final_coastal_protection_area, output_file, layer_name, "GPKG")

final_coastal_protection_coastline = flood_protection_coastline
layer_name = "final_coastal_protection_edges"
add_Layer_to_File(final_coastal_protection_coastline, output_file, layer_name, "GPKG")

